In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# Data preparation
train_data_dir = 'D:/ICME2018_Occluded-Person-Reidentification_datasets-master/ICME2018_Occluded-Person-Reidentification_datasets-master/Occ_Small_Dataset/train'  # Update this path
test_data_dir = 'D:/ICME2018_Occluded-Person-Reidentification_datasets-master/ICME2018_Occluded-Person-Reidentification_datasets-master/Occ_Small_Dataset/test'    # Update this path
input_shape = (150, 150, 3)
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical'  # Assuming you have multiple classes (persons)
)

test_datagen = ImageDataGenerator(rescale=1./255)

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'D:\\ICME2018_Occluded-Person-Reidentification_datasets-master\\ICME2018_Occluded-Person-Reidentification_datasets-master\\Occ_Small_Dataset\train'

In [ ]:
# Convert labels to one-hot encoding for test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for one-hot encoding
    shuffle=False  # Important: Do not shuffle test data
)
num_classes = train_generator.num_classes


In [ ]:
# CNN model construction
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # num_classes is the number of persons

# Compile the model with accuracy as the metric
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_generator, steps_per_epoch=train_generator.n // batch_size, epochs=1)

In [ ]:
# Model evaluation
evaluation = model.evaluate(test_generator, steps=len(test_generator))
test_loss, test_accuracy = evaluation[0], evaluation[1]

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Image plotting
def plot_person_images(person_id):
    occluded_images = []
    full_body_images = []
    predictions = []
    
    for i in range(len(test_generator)):
        batch = test_generator[i]
        images, labels = batch
        predicted_classes = model.predict(images)
        
        occluded_indices = np.where(labels[:, person_id] == 1)[0]
        occluded_images.extend(images[occluded_indices])
        full_body_indices = np.where(labels[:, person_id] == 0)[0]
        full_body_images.extend(images[full_body_indices])
        predictions.extend(np.argmax(predicted_classes, axis=1))
        
    plt.figure(figsize=(18, 8))
    for i in range(len(occluded_images)):
        plt.subplot(2, len(occluded_images), i + 1)
        plt.imshow(occluded_images[i])
        plt.title(f'Occluded Image\nPredicted: Person {predictions[i]}')
        plt.axis('off')
        
        plt.subplot(2, len(full_body_images), len(full_body_images) + i + 1)
        plt.imshow(full_body_images[i])
        plt.title(f'Full Body Image\nPredicted: Person {predictions[i]}')
        plt.axis('off')
    plt.show()

# Plot images for a random person ID
random_person_id = np.random.randint(0, num_classes)  # Choose a random person ID
plot_person_images(random_person_id)